In [ ]:
def getOtherClustersKeys(keywords, other_clusters):
    "Gets the keys for all clusters passed in the second parameter"
    
    uniqueKeywords = {}
    for i in other_clusters:
        uniqueKeywords = set(uniqueKeywords).union(set(keywords[i]))
    
    return uniqueKeywords

def getUniqueKeys(clustersNumber, keywords):
    "Used for label creation"
    
    unique_keys={}
    for cluster in range(clustersNumber):   
        other_clusters=list(set(range(clustersNumber))-set([cluster]))
        #keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
        keys_other_clusters=getOtherClustersKeys(keywords, other_clusters)
        unique=set(keywords[cluster])-keys_other_clusters
        unique_keys[cluster]=nlargest(10, unique, key=counts[cluster].get)

    return unique_keys

In [ ]:
def getInputs(filePath):
    "This function parses the csv at filepath and returns a list of sentences"
    
    inputs = []
    with open(filePath, 'r') as readFile: 
        for line in enumerate(readFile):
            inputs += line

    curatedInputs = inputs[1::2]
    n = len(curatedInputs) -1
    for i in range(0, n):
        word = curatedInputs[i]
        word = word[0:len(word)-1]
        curatedInputs[i] = word
    
        return curatedInputs

In [ ]:
filePath = "output.csv"
curatedInputs = getInputs(filePath)

In [ ]:
#vectorize 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5,min_df=1,stop_words='english')
vectorizerFitting = vectorizer.fit_transform(curatedInputs)

In [ ]:
#make clusters
from sklearn.cluster import KMeans
clustersNumber = 20
km = KMeans(n_clusters = clustersNumber, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(vectorizerFitting)



In [ ]:
#cluster numbering and total words
import numpy as np
np.unique(km.labels_, return_counts=True)

In [ ]:
#assign cluster data
text={}
for i,cluster in enumerate(km.labels_):
    oneDocument = curatedInputs[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [ ]:
#cleanup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk 

_stopwords = set(stopwords.words('english') + list(punctuation) + ["''","``","/i","1","2","3","4","5","6","7","8","9","0" ])
                 
keywords={}
counts={}
for cluster in range(clustersNumber):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq

In [ ]:
#final output of type key,clusterData                
uniqueKeys = getUniqueKeys(clustersNumber, keywords)


In [ ]:
def getRestrictedLabels(usableLabels, kmLabels):
    "Gets a new km.labels_ like object, that can be used to k-nearest classify an input between some but not all the clusters"
        
    kmLabels = kmLabels.copy()
    for i in range(len(kmLabels)):
        label = kmLabels[i]
        if label not in usableLabels:
            kmLabels[i] = -1
        
    return kmLabels

def getShrinkedCorpus(restrictedLabels, corpus):
    "Shrinks a corpus to relevant labels"
    
    restrictedCorpus = []
    for i in range(len(restrictedLabels)):
        if restrictedLabels[i] != -1:
            restrictedCorpus += corpus[i]
    
    return restrictedCorpus

def getShrinkedLabels(restrictedLabels):
    "Shrinks the label list to match the shrinkatinated corpus"
    
    shrinkedLabels = []
    for i in range(len(restrictedLabels)):
        if restrictedLabels[i] != -1:
            shrinkedLabels += restrictedLabels[i]
    
    return shrinkedLabels
     

In [ ]:
usableLabels = [1, 2, 4, 6]
restrictedLabels = getRestrictedLabels(usableLabels, km.labels_)
shrinkedCorpus = getShrinkedCorpus(restrictedLabels, vectorizerFitting)
shrinkedLabels = getShrinkedLabels(restrictedLabels)

In [2]:
class KnearestWrapper:
    
    def __init__(self, pathToSavedModel, pathToSavedVectorizerFitting, pathToSavedEdgesToRoomsDicti, pathToSavedRoomsToBiasDict):
        from sklearn.neighbors import KNeighborsClassifier
        self.__model = [] # @TODO Alex get these please
        self.__fitting = [] 
        self.__biasDict = {}
        self.__RoomToEdgesDict = {}
    
    def __getBias(self, roomId):
        return " "
        # emptyString

    def __getEdges(self, roomId):
    
        #modify this function to look up in the edge dictionary
        #this is now hard coded!
        edges = []
        edge1 = {}
        edge1["Destination"] = 1
        edge1["Labels"] = [1]
    
        edges += edge1
    
        edge2 = {}
        edge2["Destination"] = 2
        edge3["Labels"] = [2, 4]
    
        edges += edge2
    
        edge3 = {}
        edge3["Destination"] = 3
        edge3["Labels"] = [3,5]
    
        edges += edge3 
    
        return edges


    def __getAllLabels(edges):
        "Gets all labels within referenced in a list of edges"
    
        allLabels = []
        for edge in edges:
            allLabels += edge["Labels"]
    
        return allLabels
    
    def __getUnifiedLabels(shrinkedLabels, edges):
        "Returns a labeling model where each cluster is actually a room"
    
        labelToRoomDict = {}
    
        for edge in edges:
            for label in edge["Labels"]:
                labelToRoomDict[label] = edge["Destination"]
    
        unifiedLabels = []
        for label in shrinkedLabels:
            unifiedLabels.append(labelToRoomDict[label])
        
        return unifiedLabels



    def modifiedKmeans(self, playerInput, roomId):
        "Returns the id of the room it goes to"
    
        bias = getBias(roomId)
        playerInput = playerInput + bias #string + i assume
    
        edges = getEdges(roomId)
    
        usableLabels = getUsableLables[edges]
    
        restrictedLabels = getRestrictedLabels(usableLabels, allLables)
        shrinkedCorpus = getRestrictedCorpus(restrictedLabels, corpus)
        shrinkedLabels = getShrinkedLabels(restrictedLabels)
    
        unifiedLabels = getUnifiedLabels(shrinkedLabels, edges)
    
        classifier = KNeighborsClassifier(n_neighbors=5)
        classifier.fit(shrinkedCorpus,unifiedLabels)
    
        return classifier.predict(playerInput)
         